In [34]:
import numpy as np

In [35]:
# quick and dirty random bounding boxes
def get_boxes(num_boxes, img_width, img_height, min_box_width, min_box_height, max_box_width, max_box_height, seed=0):
    rng = np.random.default_rng(seed)
    max_half_box_width = max_box_width / 2
    max_half_box_height = max_box_height / 2

    centers_x = rng.random(size=num_boxes, dtype=np.float32) * (img_width - max_box_width) + max_half_box_width
    centers_y = rng.random(size=num_boxes, dtype=np.float32) * (img_height - max_box_height) + max_half_box_height
    widths = rng.random(size=num_boxes, dtype=np.float32) * (max_box_width - min_box_width) + min_box_width
    heights = rng.random(size=num_boxes, dtype=np.float32) * (max_box_height - min_box_height) + min_box_height

    corners_x1 = centers_x - (widths / 2)
    corners_y1 = centers_y - (heights / 2)
    corners_x2 = img_width - widths
    corners_y2 = img_height - heights

    corners_x1 = centers_x - (widths / 2)
    corners_y1 = centers_y - (heights / 2)
    corners_x2 = centers_x + (widths / 2)
    corners_y2 = centers_y + (heights / 2)

    return np.round(np.array(list(zip(corners_x1, corners_y1, corners_x2, corners_y2))))


# non-vectorized IoU
def get_iou(box1, box2):
    box1_x1, box1_y1, box1_x2, box1_y2 = box1
    box2_x1, box2_y1, box2_x2, box2_y2 = box2

    box1_delta_x = (box1_x2 - box1_x1)
    box1_delta_y = (box1_y2 - box1_y1)
    box2_delta_x = (box2_x2 - box2_x1)
    box2_delta_y = (box2_y2 - box2_y1)

    x_overlap = max(0, min(box1_delta_x, box2_delta_x, box1_x2 - box2_x1, box2_x2 - box1_x1))
    y_overlap = max(0, min(box1_delta_y, box2_delta_y, box1_y2 - box2_y1, box2_y2 - box1_y1))

    intersection_area = x_overlap * y_overlap

    box1_area = box1_delta_x * box1_delta_y
    box2_area = box2_delta_x * box2_delta_y

    union_area = box1_area + box2_area - intersection_area
    iou = intersection_area / union_area

    return iou


# vectorized IoU
def get_iou_matrix1(boxes_1, boxes_2):
    n = boxes_1.shape[0]
    m = boxes_2.shape[0]

    boxes_1_x1, boxes_1_y1 = boxes_1[:, 0].reshape(-1, 1), boxes_1[:, 1].reshape(-1, 1)
    boxes_1_x2, boxes_1_y2 = boxes_1[:, 2].reshape(-1, 1), boxes_1[:, 3].reshape(-1, 1)
    boxes_2_x1, boxes_2_y1 = boxes_2[:, 0].reshape(-1, 1), boxes_2[:, 1].reshape(-1, 1)
    boxes_2_x2, boxes_2_y2 = boxes_2[:, 2].reshape(-1, 1), boxes_2[:, 3].reshape(-1, 1)

    boxes_1_delta_x = (boxes_1_x2 - boxes_1_x1)
    boxes_1_delta_y = (boxes_1_y2 - boxes_1_y1)
    boxes_2_delta_x = (boxes_2_x2 - boxes_2_x1)
    boxes_2_delta_y = (boxes_2_y2 - boxes_2_y1)
    boxes_1_area = boxes_1_delta_x * boxes_1_delta_y
    boxes_2_area = boxes_2_delta_x * boxes_2_delta_y

    x_overlap_matrix = np.zeros((4, n, m))
    x_overlap_matrix[0] = np.repeat(boxes_1_delta_x, m, axis=1)
    x_overlap_matrix[1] = np.repeat(boxes_2_delta_x.T, n, axis=0)
    x_overlap_matrix[2] = np.repeat(boxes_1_x2, m, axis=1) - np.repeat(boxes_2_x1.T, n, axis=0)
    x_overlap_matrix[3] = np.repeat(boxes_2_x2.T, n, axis=0) - np.repeat(boxes_1_x1, m, axis=1)
    x_overlap_matrix = np.clip(np.min(x_overlap_matrix, axis=0), 0, None)

    y_overlap_matrix = np.zeros((4, n, m))
    y_overlap_matrix[0] = np.repeat(boxes_1_delta_y, m, axis=1)
    y_overlap_matrix[1] = np.repeat(boxes_2_delta_y.T, n, axis=0)
    y_overlap_matrix[2] = np.repeat(boxes_1_y2, m, axis=1) - np.repeat(boxes_2_y1.T, n, axis=0)
    y_overlap_matrix[3] = np.repeat(boxes_2_y2.T, n, axis=0) - np.repeat(boxes_1_y1, m, axis=1)
    y_overlap_matrix = np.clip(np.min(y_overlap_matrix, axis=0), 0, None)

    intersection_area_matrix = x_overlap_matrix * y_overlap_matrix
    boxes_combined_area_matrix = np.repeat(boxes_1_area, m, axis=1) + np.repeat(boxes_2_area.T, n, axis=0)

    union_area_matrix = boxes_combined_area_matrix - intersection_area_matrix
    iou_matrix = intersection_area_matrix / union_area_matrix

    return iou_matrix


# vectorized IoU using masked arrays
def get_iou_matrix2(boxes_1, boxes_2):
    n = boxes_1.shape[0]
    m = boxes_2.shape[0]

    boxes_1_x1, boxes_1_y1 = boxes_1[:, 0].reshape(-1, 1), boxes_1[:, 1].reshape(-1, 1)
    boxes_1_x2, boxes_1_y2 = boxes_1[:, 2].reshape(-1, 1), boxes_1[:, 3].reshape(-1, 1)
    boxes_2_x1, boxes_2_y1 = boxes_2[:, 0].reshape(-1, 1), boxes_2[:, 1].reshape(-1, 1)
    boxes_2_x2, boxes_2_y2 = boxes_2[:, 2].reshape(-1, 1), boxes_2[:, 3].reshape(-1, 1)

    boxes_1_delta_x = (boxes_1_x2 - boxes_1_x1)
    boxes_1_delta_y = (boxes_1_y2 - boxes_1_y1)
    boxes_2_delta_x = (boxes_2_x2 - boxes_2_x1)
    boxes_2_delta_y = (boxes_2_y2 - boxes_2_y1)
    boxes_1_area = boxes_1_delta_x * boxes_1_delta_y
    boxes_2_area = boxes_2_delta_x * boxes_2_delta_y

    x_overlap_matrix = np.zeros((4, n, m))
    x_overlap_matrix[0] = np.repeat(boxes_1_delta_x, m, axis=1)
    x_overlap_matrix[1] = np.repeat(boxes_2_delta_x.T, n, axis=0)
    x_overlap_matrix[2] = np.repeat(boxes_1_x2, m, axis=1) - np.repeat(boxes_2_x1.T, n, axis=0)
    x_overlap_matrix[3] = np.repeat(boxes_2_x2.T, n, axis=0) - np.repeat(boxes_1_x1, m, axis=1)
    x_overlap_matrix = np.min(x_overlap_matrix, axis=0)
    x_overlap_mask = (x_overlap_matrix <= 0)

    y_overlap_matrix = np.zeros((4, n, m))
    y_overlap_matrix[0] = np.repeat(boxes_1_delta_y, m, axis=1)
    y_overlap_matrix[1] = np.repeat(boxes_2_delta_y.T, n, axis=0)
    y_overlap_matrix[2] = np.repeat(boxes_1_y2, m, axis=1) - np.repeat(boxes_2_y1.T, n, axis=0)
    y_overlap_matrix[3] = np.repeat(boxes_2_y2.T, n, axis=0) - np.repeat(boxes_1_y1, m, axis=1)
    y_overlap_matrix = np.min(y_overlap_matrix, axis=0)
    y_overlap_mask = (y_overlap_matrix <= 0)

    xy_overlap_mask = x_overlap_mask | y_overlap_mask
    x_overlap_matrix = np.ma.array(x_overlap_matrix, mask=xy_overlap_mask)
    y_overlap_matrix = np.ma.array(y_overlap_matrix, mask=xy_overlap_mask)

    intersection_area_matrix = x_overlap_matrix * y_overlap_matrix
    boxes_combined_area_matrix = np.ma.array(np.repeat(boxes_1_area, m, axis=1) + np.repeat(boxes_2_area.T, n, axis=0), mask=xy_overlap_mask)

    union_area_matrix = boxes_combined_area_matrix - intersection_area_matrix
    iou_matrix = intersection_area_matrix / union_area_matrix

    return iou_matrix

In [36]:
IMG_WIDTH = 1000
IMG_HEIGHT = 1000

## Timing 200 x 200 bounding boxes
Executing the nested loops version takes a veeeeery long time so a small 200 x 200 sample was tested using this unoptimized approach and its execution time compared to averaged execution times for each of the two optimized approaches.

### Case 1 - dense 200 x 200
All boxes overlapping.

In [37]:
NUM_BOXES_IN_SET1 = 200
NUM_BOXES_IN_SET2 = 200

MIN_BOX_WIDTH = 700
MIN_BOX_HEIGHT = 700
MAX_BOX_WIDTH = 800
MAX_BOX_HEIGHT = 800

boxes_1 = get_boxes(NUM_BOXES_IN_SET1, IMG_WIDTH, IMG_HEIGHT, MIN_BOX_WIDTH, MIN_BOX_HEIGHT, MAX_BOX_WIDTH, MAX_BOX_HEIGHT, seed=0)
boxes_2 = get_boxes(NUM_BOXES_IN_SET1, IMG_WIDTH, IMG_HEIGHT, MIN_BOX_WIDTH, MIN_BOX_HEIGHT, MAX_BOX_WIDTH, MAX_BOX_HEIGHT, seed=1)

#### Nested loops version

In [38]:
%%time

iou_matrix_dense = np.zeros((NUM_BOXES_IN_SET1, NUM_BOXES_IN_SET2))

for n, box1 in enumerate(boxes_1):
    for m, box2 in enumerate(boxes_2):
        iou_matrix_dense[n, m] = get_iou(box1, box2)

CPU times: user 428 ms, sys: 3.86 ms, total: 432 ms
Wall time: 435 ms


#### Vectorized version 1

In [39]:
%%timeit

get_iou_matrix1(boxes_1, boxes_2)

100 loops, best of 5: 3.19 ms per loop


In [40]:
iou_matrix1_dense = get_iou_matrix1(boxes_1, boxes_2)

#### Vectorized version using masked arrays

In [41]:
%%timeit

get_iou_matrix2(boxes_1, boxes_2)

100 loops, best of 5: 3.73 ms per loop


In [42]:
iou_matrix2_dense = get_iou_matrix2(boxes_1, boxes_2)

##### Equality of the three dense 200 x 200 matrices

In [43]:
np.allclose(iou_matrix_dense, iou_matrix1_dense, iou_matrix2_dense)

True

##### Sparsity of the dense 200 x 200 matrix

In [44]:
print(f'sparsity = {np.sum((iou_matrix_dense == 0))/iou_matrix_dense.size:.1%}')

sparsity = 0.0%


### Case 2 - sparse 200 x 200
Most boxes non-overlapping.

In [45]:
NUM_BOXES_IN_SET1 = 200
NUM_BOXES_IN_SET2 = 200

MIN_BOX_WIDTH = 70
MIN_BOX_HEIGHT = 70
MAX_BOX_WIDTH = 80
MAX_BOX_HEIGHT = 80

boxes_1 = get_boxes(NUM_BOXES_IN_SET1, IMG_WIDTH, IMG_HEIGHT, MIN_BOX_WIDTH, MIN_BOX_HEIGHT, MAX_BOX_WIDTH, MAX_BOX_HEIGHT, seed=0)
boxes_2 = get_boxes(NUM_BOXES_IN_SET1, IMG_WIDTH, IMG_HEIGHT, MIN_BOX_WIDTH, MIN_BOX_HEIGHT, MAX_BOX_WIDTH, MAX_BOX_HEIGHT, seed=1)

#### Nested loops version

In [46]:
%%time

iou_matrix_sparse = np.zeros((NUM_BOXES_IN_SET1, NUM_BOXES_IN_SET2))

for n, box1 in enumerate(boxes_1):
    for m, box2 in enumerate(boxes_2):
        iou_matrix_sparse[n, m] = get_iou(box1, box2)

CPU times: user 608 ms, sys: 10 µs, total: 608 ms
Wall time: 612 ms


#### Vectorized version 1

In [47]:
%%timeit

get_iou_matrix1(boxes_1, boxes_2)

100 loops, best of 5: 3.33 ms per loop


In [48]:
iou_matrix1_sparse = get_iou_matrix1(boxes_1, boxes_2)

#### Vectorized version using masked arrays

In [49]:
%%timeit

get_iou_matrix2(boxes_1, boxes_2)

100 loops, best of 5: 4.1 ms per loop


In [50]:
iou_matrix2_sparse = get_iou_matrix2(boxes_1, boxes_2)

##### Equality of the three sparse 200 x 200 matrices

In [51]:
np.allclose(iou_matrix_sparse, iou_matrix1_sparse, iou_matrix2_sparse)

True

##### Sparsity of the dense 200 x 200 matrix

In [52]:
print(f'sparsity = {np.sum((iou_matrix_sparse == 0))/iou_matrix_sparse.size:.2%}')

sparsity = 97.59%


## Timing 2000 x 2000 bounding boxes

### Case 1 - dense 2000 x 2000
All boxes overlapping.

In [53]:
NUM_BOXES_IN_SET1 = 2000
NUM_BOXES_IN_SET2 = 2000

MIN_BOX_WIDTH = 700
MIN_BOX_HEIGHT = 700
MAX_BOX_WIDTH = 800
MAX_BOX_HEIGHT = 800

boxes_1 = get_boxes(NUM_BOXES_IN_SET1, IMG_WIDTH, IMG_HEIGHT, MIN_BOX_WIDTH, MIN_BOX_HEIGHT, MAX_BOX_WIDTH, MAX_BOX_HEIGHT, seed=0)
boxes_2 = get_boxes(NUM_BOXES_IN_SET1, IMG_WIDTH, IMG_HEIGHT, MIN_BOX_WIDTH, MIN_BOX_HEIGHT, MAX_BOX_WIDTH, MAX_BOX_HEIGHT, seed=1)

#### Vectorized version 1

In [54]:
%time iou_matrix1_full_dense = get_iou_matrix1(boxes_1, boxes_2)

CPU times: user 401 ms, sys: 24 ms, total: 425 ms
Wall time: 429 ms


In [55]:
iou_matrix1_full_dense = get_iou_matrix1(boxes_1, boxes_2)

#### Vectorized version using masked arrays

In [56]:
%time iou_matrix2_full_dense = get_iou_matrix2(boxes_1, boxes_2)

CPU times: user 452 ms, sys: 8.77 ms, total: 461 ms
Wall time: 456 ms


In [57]:
iou_matrix2_full_dense = get_iou_matrix2(boxes_1, boxes_2)

##### Equality of the two dense 2000 x 2000 matrices

In [58]:
np.allclose(iou_matrix1_full_dense, iou_matrix2_full_dense)

True

##### Sparsity of the dense 2000 x 2000 matrix

In [59]:
print(f'sparsity = {np.sum((iou_matrix1_full_dense == 0))/iou_matrix1_full_dense.size:.1%}')

sparsity = 0.0%


### Case 2 - sparse 2000 x 2000
Most boxes non-overlapping.

In [60]:
NUM_BOXES_IN_SET1 = 2000
NUM_BOXES_IN_SET2 = 2000

MIN_BOX_WIDTH = 70
MIN_BOX_HEIGHT = 70
MAX_BOX_WIDTH = 80
MAX_BOX_HEIGHT = 80

boxes_1 = get_boxes(NUM_BOXES_IN_SET1, IMG_WIDTH, IMG_HEIGHT, MIN_BOX_WIDTH, MIN_BOX_HEIGHT, MAX_BOX_WIDTH, MAX_BOX_HEIGHT, seed=0)
boxes_2 = get_boxes(NUM_BOXES_IN_SET1, IMG_WIDTH, IMG_HEIGHT, MIN_BOX_WIDTH, MIN_BOX_HEIGHT, MAX_BOX_WIDTH, MAX_BOX_HEIGHT, seed=1)

#### Vectorized version 1

In [61]:
%time iou_matrix1_full_sparse = get_iou_matrix1(boxes_1, boxes_2)

CPU times: user 428 ms, sys: 3.9 ms, total: 431 ms
Wall time: 430 ms


In [62]:
iou_matrix1_full_sparse = get_iou_matrix1(boxes_1, boxes_2)

#### Vectorized version using masked arrays

In [63]:
%time iou_matrix2_full_sparse = get_iou_matrix2(boxes_1, boxes_2)

CPU times: user 469 ms, sys: 8.96 ms, total: 478 ms
Wall time: 479 ms


In [64]:
iou_matrix2_full_sparse = get_iou_matrix2(boxes_1, boxes_2)

##### Equality of the two sparse 2000 x 2000 matrices

In [65]:
np.allclose(iou_matrix1_full_sparse, iou_matrix2_full_sparse)

True

##### Sparsity of the sparse 2000 x 2000 matrix

In [66]:
print(f'sparsity = {np.sum((iou_matrix1_full_sparse == 0))/iou_matrix1_full_sparse.size:.2%}')

sparsity = 97.60%
